In [3]:
import sqlite3
conn = sqlite3.connect('lrml_editor.db')  # Connects to or creates the example.db file

In [4]:
cursor = conn.cursor()

# Create table in SQLite
cursor.execute('''
CREATE TABLE IF NOT EXISTS UNICLASS (
    KEY TEXT,
    CODE TEXT,
    DESCRIPTION TEXT,
    TABLE_NAME TEXT,
    TEXT TEXT,
    TITLE TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS ANALYTICS (
    FUNC TEXT,
    VALUE TEXT,
    FILE_NAME TEXT,
    CLAUSE_NUMBER TEXT,
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    AUTHOR TEXT
)
''')

# Create table in SQLite
cursor.execute('''
CREATE TABLE IF NOT EXISTS BUVO (
    KEY TEXT,
    ATOM_NAME TEXT,
    DESCRIPTION TEXT,
    UPDATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    REFERENCE TEXT,
    TEXT TEXT,
    ID INTEGER PRIMARY KEY AUTOINCREMENT
)
''')

# Create unique index in SQLite
cursor.execute('''
CREATE UNIQUE INDEX IF NOT EXISTS BUVO_PK ON BUVO (KEY)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS FUVO (
    KEY TEXT,
    ATOM_NAME TEXT,
    DESCRIPTION TEXT,
    UPDATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    REFERENCE TEXT,
    TEXT TEXT,
    VARS TEXT,
    ID INTEGER PRIMARY KEY AUTOINCREMENT
)
''')

# Create unique index on ID in SQLite
cursor.execute('''
CREATE UNIQUE INDEX IF NOT EXISTS FUVO_PK ON FUVO (ID)
''')

# Create unique index on KEY in SQLite
cursor.execute('''
CREATE UNIQUE INDEX IF NOT EXISTS UK_KEY_FUVO ON FUVO (KEY)
''')

# Create table in SQLite
cursor.execute('''
CREATE TABLE IF NOT EXISTS IFC (
    KEY TEXT,
    CODE TEXT,
    DESCRIPTION TEXT,
    TEXT TEXT,
    TITLE TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS IFC_PROPS (
    KEY TEXT,
    CODE TEXT,
    DESCRIPTION TEXT,
    TEXT TEXT,
    TITLE TEXT,
    DATATYPE TEXT,
    PROPERTYSET TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS LOVO (
    KEY TEXT,
    ATOM_NAME TEXT,
    DESCRIPTION TEXT,
    UPDATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    REFERENCE TEXT,
    TEXT TEXT,
    ID INTEGER PRIMARY KEY AUTOINCREMENT
)
''')

# Create unique index on KEY in SQLite
cursor.execute('''
CREATE UNIQUE INDEX IF NOT EXISTS LOVO_PK ON LOVO (KEY)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS LRML (
    FILE_NAME TEXT,
    CLAUSE_NUMBER TEXT,
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    ID INTEGER PRIMARY KEY AUTOINCREMENT
)
''')

# Create unique index on ID in SQLite
cursor.execute('''
CREATE UNIQUE INDEX IF NOT EXISTS LRML_PK ON LRML (ID)
''')

# Create unique index on FILE_NAME and CLAUSE_NUMBER in SQLite
cursor.execute('''
CREATE UNIQUE INDEX IF NOT EXISTS NEW_UK_1 ON LRML (FILE_NAME, CLAUSE_NUMBER)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS LRML_UPDATES (
    LRML_ID INTEGER,
    TEXT TEXT,
    LRML TEXT,
    UPDATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    PARAPHRASE TEXT,
    COMMENTS TEXT,
    AUTHOR TEXT
)
''')

# Create regular index on TEXT in SQLite
cursor.execute('''
CREATE INDEX IF NOT EXISTS TEXT_IDX ON LRML_UPDATES (TEXT)
''')

# Create unique index on ID in SQLite
cursor.execute('''
CREATE UNIQUE INDEX IF NOT EXISTS LRML_UPDATES_PK ON LRML_UPDATES (ID)
''')

# Create table in SQLite
cursor.execute('''
CREATE TABLE IF NOT EXISTS OMNICLASS (
    KEY TEXT,
    CODE TEXT,
    DESCRIPTION TEXT,
    TABLE_NAME TEXT,
    TEXT TEXT,
    TITLE TEXT,
    TABLE_NUMBER TEXT,
    SYNONYMS TEXT
)
''')

# Create view in SQLite
cursor.execute('''
CREATE VIEW IF NOT EXISTS LRML_VIEW AS 
SELECT 
    LRML.ID, 
    LRML.FILE_NAME, 
    LRML.CLAUSE_NUMBER, 
    LRML.CREATED_AT, 
    LRML_UPDATES.ID AS UPDATE_ID, 
    LRML_UPDATES.TEXT, 
    LRML_UPDATES.LRML, 
    LRML_UPDATES.PARAPHRASE, 
    LRML_UPDATES.COMMENTS, 
    LRML_UPDATES.UPDATED_AT
FROM LRML
LEFT OUTER JOIN (
    SELECT LRML_ID, MAX(UPDATED_AT) AS MAX_UPDATED_AT
    FROM LRML_UPDATES
    GROUP BY LRML_ID
) latest
ON LRML.ID = latest.LRML_ID
LEFT OUTER JOIN LRML_UPDATES
ON LRML_UPDATES.LRML_ID = latest.LRML_ID
AND LRML_UPDATES.UPDATED_AT = latest.MAX_UPDATED_AT
ORDER BY LRML.ID
''')

# Commit and close connection
conn.commit()
cursor.close()
conn.close()

In [10]:
import sqlite3
import os
import pandas as pd

# Connect to SQLite
conn = sqlite3.connect('data/lrml_editor.db')
cursor = conn.cursor()

# Directory containing the CSV files
data_dir = 'data2'

# List of tables and corresponding CSV filenames
tables = ['uniclass', 'analytics', 'buvo', 'fuvo', 'ifc', 'ifc_props', 'lovo', 'lrml', 'lrml_updates']

# Function to load CSV data into SQLite
def load_csv_to_sqlite(table_name, file_path):
    # Read CSV file into DataFrame
    df = pd.read_csv(file_path)
    # Drop any 'Unnamed' columns
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    # Write DataFrame to SQLite table
    df.to_sql(table_name.upper(), conn, if_exists='append', index=False)

# Iterate over tables and load corresponding CSV files
for table in tables:
    csv_file = os.path.join(data_dir, f'{table}.csv')
    if os.path.isfile(csv_file):
        load_csv_to_sqlite(table, csv_file)
    else:
        print(f"CSV file for table {table} not found.")

# Commit and close connection
conn.commit()
conn.close()
